# Neural Sparse Search Tutorial
![Course](../../static_images/ai_ml_search_opensearch_intermediate.jpeg)

## Overview
Semantic search relies on dense retrieval that is based on text embedding models. However, dense methods use k-NN search, which consumes a large amount of memory and CPU resources. An alternative to semantic search, neural sparse search is implemented using an inverted index and is thus as efficient as BM25.

Neural Sparse Search combines the efficiency of sparse retrieval with the relevance of neural embeddings. This notebook demonstrates various use cases and implementation techniques.

To further boost search relevance, you can combine neural sparse search with dense semantic search using a hybrid query

```mermaid
graph TB
    A["🧠 Neural Sparse Search Workflow"] --> B{"Search Mode"}
    B -->|"Doc-Only Mode"| C["📄 Sparse Encoding at Ingestion"]
    B -->|"Bi-Encoder Mode"| D["🔄 Encoding at Both Stages"]
    B -->|"ANN Mode"| E["⚡ Approximate Nearest Neighbor"]
    
    C --> F["🔑 Token-Weight Pairs"]
    D --> G["🔍 Query Text Encoding"]
    E --> H["📊 Clustered Inverted Index"]
    
    F --> I["✅ Fast Retrieval"]
    G --> J["🎯 High Relevance"]
    H --> K["⚙️ Scalable Performance"]
    
    I --> L["🎁 Search Results"]
    J --> L
    K --> L
    
    M["🔧 Ingest Pipeline"] --> C
    N["🗂️ Rank Features Index"] --> C
    O["🎨 Semantic Field"] --> C
    
    style A fill:#FF6B6B,stroke:#333,stroke-width:3px,color:#fff
    style C fill:#4ECDC4,stroke:#333,stroke-width:2px,color:#fff
    style D fill:#45B7D1,stroke:#333,stroke-width:2px,color:#fff
    style E fill:#96CEB4,stroke:#333,stroke-width:2px,color:#fff
    style L fill:#FFEAA7,stroke:#333,stroke-width:3px,color:#333
    style I fill:#DDA15E,stroke:#333,stroke-width:2px,color:#fff
    style J fill:#BC6C25,stroke:#333,stroke-width:2px,color:#fff
    style K fill:#8E44AD,stroke:#333,stroke-width:2px,color:#fff
    style M fill:#E74C3C,stroke:#333,stroke-width:2px,color:#fff
    style N fill:#3498DB,stroke:#333,stroke-width:2px,color:#fff
    style O fill:#9B59B6,stroke:#333,stroke-width:2px,color:#fff
```

## Prerequisites & Setup

This notebook requires:
- OpenSearch cluster (local or remote)
- Python 3.8+
- Required libraries: opensearch-py, numpy, pandas

## 🐳 Docker Setup
- **If docker compose up fails , start it manually from shell**

In [35]:
%%bash
cd ../
echo "🚀 Starting fully optimized OpenSearch cluster..."

# Start the optimized cluster
docker compose -f docker-compose-fully-optimized.yml down -v
docker compose -f docker-compose-fully-optimized.yml up -d

# Wait for startup
echo "⏳ Waiting for cluster to initialize..."
sleep 60

# Check cluster health
echo "🏥 Checking cluster health..."
curl -k -u admin:Developer@123 https://localhost:9200/_cluster/health?pretty

🚀 Starting fully optimized OpenSearch cluster...


 Network 4ai_search_opensearch-net Creating 
 Network 4ai_search_opensearch-net Created 
 Volume 4ai_search_opensearch-optimized-data2 Creating 
 Volume 4ai_search_opensearch-optimized-data2 Created 
 Volume 4ai_search_opensearch-optimized-data1 Creating 
 Volume 4ai_search_opensearch-optimized-data1 Created 
 Container opensearch-optimized-node1 Creating 
 Container opensearch-optimized-dashboards Creating 
 Container opensearch-optimized-node2 Creating 
 Container opensearch-optimized-dashboards Created 
 Container opensearch-optimized-node2 Created 
 Container opensearch-optimized-node1 Created 
 Container opensearch-optimized-node1 Starting 
 Container opensearch-optimized-node2 Starting 
 Container opensearch-optimized-dashboards Starting 
 Container opensearch-optimized-node1 Started 
 Container opensearch-optimized-node2 Started 
 Container opensearch-optimized-dashboards Started 


⏳ Waiting for cluster to initialize...
🏥 Checking cluster health...


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   550  100   550    0     0   2170      0 --:--:-- --:--:-- --:--:--  2173


{
  "cluster_name" : "opensearch-optimized-cluster",
  "status" : "green",
  "timed_out" : false,
  "number_of_nodes" : 2,
  "number_of_data_nodes" : 2,
  "discovered_master" : true,
  "discovered_cluster_manager" : true,
  "active_primary_shards" : 4,
  "active_shards" : 8,
  "relocating_shards" : 0,
  "initializing_shards" : 0,
  "unassigned_shards" : 0,
  "delayed_unassigned_shards" : 0,
  "number_of_pending_tasks" : 0,
  "number_of_in_flight_fetch" : 0,
  "task_max_waiting_in_queue_millis" : 0,
  "active_shards_percent_as_number" : 100.0
}


In [36]:
# Import required libraries
from opensearchpy import OpenSearch
import sys, os
from opensearchpy.helpers import bulk
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import json
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## Step 1: Configure OpenSearch Client

Initialize connection to OpenSearch cluster with proper authentication.

In [37]:
# Get the current working directory of the notebook
current_dir = os.getcwd()

DATA_DIR = os.path.abspath(os.path.join(current_dir, '../../0. DATA'))

# Construct the path to the directory levels up
module_paths = [os.path.abspath(os.path.join(current_dir, '../../')),]

# Add the module path to sys.path if it's not already there
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

try:
    import helpers as hp
    print(f"✅ Helper module loaded from: {module_paths[0]}")
except ImportError as e:
    print(f"⚠️ Helper module not found: {e}")

# Configuration
IS_AUTH = True
HOST = 'localhost'

print(f"📍 Data directory: {DATA_DIR}")
print(f"🔌 Connecting to OpenSearch at {HOST}:9200")

✅ Helper module loaded from: /data/OPENSEARCH_INTERMEDIATE_TUTORIAL
📍 Data directory: /data/OPENSEARCH_INTERMEDIATE_TUTORIAL/0. DATA
🔌 Connecting to OpenSearch at localhost:9200


In [38]:
# Initialize the OpenSearch client
if IS_AUTH:
    client = OpenSearch(
        hosts=[{'host': HOST, 'port': 9200}],
        http_auth=('admin', 'Developer@123'),  # Replace with your credentials
        use_ssl=True,
        verify_certs=False,
        ssl_show_warn=False
    )
else:
    client = OpenSearch(
        hosts=[{'host': HOST, 'port': 9200}],
        use_ssl=False,
        verify_certs=False,
        ssl_assert_hostname=False,
        ssl_show_warn=False
    )

# Verify connection
try:
    info = client.info()
    print(f"✅ Connected to {info['version']['distribution']} v{info['version']['number']}")
    print(f"📊 Cluster Status: {client.cluster.health()['status']}")
except Exception as e:
    print(f"❌ Connection failed: {e}")
    raise

✅ Connected to opensearch v3.3.0
📊 Cluster Status: green


## Step 2: Register and Deploy Sparse Encoding Model

### Overview
Neural sparse search requires a sparse encoding model to generate sparse vector embeddings from text. This model converts text into token-weight pairs.

In [39]:
# Step 2a: Register a model group
model_group_name = f"neural_sparse_models_{int(time.time())}"
print(f"📦 Registering model group: {model_group_name}")

try:
    model_group_response = client.transport.perform_request(
        method='POST',
        url='/_plugins/_ml/model_groups/_register',
        body={
            "name": model_group_name,
            "description": "Model group for neural sparse search examples"
        }
    )
    
    model_group_id = model_group_response['model_group_id']
    print(f"✅ Model group registered with ID: {model_group_id}")
except Exception as e:
    print(f"⚠️ Model group registration error: {e}")
    model_group_id = None

📦 Registering model group: neural_sparse_models_1767453012
✅ Model group registered with ID: X0JohJsBgGjK7mcRsg8u


In [40]:
# Step 2b: Register sparse encoding model (doc-only mode)
print("\n🧠 Registering sparse encoding model...")
print("   Model: amazon/neural-sparse/opensearch-neural-sparse-encoding-doc-v3-distill")

try:
    register_response = client.transport.perform_request(
        method='POST',
        url='/_plugins/_ml/models/_register?deploy=true',
        body={
            "name": "amazon/neural-sparse/opensearch-neural-sparse-encoding-doc-v3-distill",
            "version": "1.0.0",
            "model_format": "TORCH_SCRIPT",
            "model_group_id": model_group_id if model_group_id else None
        }
    )
    
    register_task_id = register_response['task_id']
    print(f"📋 Model registration task ID: {register_task_id}")
except Exception as e:
    print(f"❌ Model registration error: {e}")
    raise


🧠 Registering sparse encoding model...
   Model: amazon/neural-sparse/opensearch-neural-sparse-encoding-doc-v3-distill
📋 Model registration task ID: YEJohJsBgGjK7mcR4A-V


In [41]:
# Step 2c: Wait for model registration to complete
print("\n⏳ Waiting for model registration to complete...")
max_attempts = 36  # e.g., 3 minutes with 5 seconds interval
attempt = 0
model_id = None

while attempt < max_attempts:
    try:
        task_status = client.transport.perform_request(
            method='GET',
            url=f'/_plugins/_ml/tasks/{register_task_id}'
        )
        
        state = task_status.get('state', 'UNKNOWN')
        print(f"   Attempt {attempt + 1}/{max_attempts}: Status = {state}", end='\r')
        
        if state == 'COMPLETED':
            model_id = task_status.get('model_id')
            print(f"\n✅ Model registered successfully! Model ID: {model_id}")
            break
        elif state == 'FAILED':
            print(f"\n❌ Model registration failed: {task_status.get('error', 'Unknown error')}")
            break
        
        time.sleep(5)
        attempt += 1
    except Exception as e:
        print(f"\n⚠️ Error checking task status: {e}")
        time.sleep(5)
        attempt += 1

if not model_id:
    print("⚠️ Model registration did not complete within timeout. Please verify manually.")


⏳ Waiting for model registration to complete...
   Attempt 7/36: Status = COMPLETED
✅ Model registered successfully! Model ID: YUJohJsBgGjK7mcR5A-o


## Step 3: Create Ingest Pipeline

### Overview
The ingest pipeline automatically generates sparse embeddings when documents are indexed. The `sparse_encoding` processor uses the registered model to convert text into token-weight pairs.

### Pruning Configuration: `prune_type` and `prune_ratio`

When generating sparse embeddings, the model produces many token-weight pairs. **Pruning** removes less important tokens to optimize storage and query performance.

#### `prune_type`
Defines the strategy for selecting which tokens to keep:
- **`max_ratio`** (Recommended): Keeps only the top tokens by weight ratio
  - For example, if you set `prune_ratio: 0.1`, it keeps only the 10% of tokens with the highest weights
  - Eliminates tokens with very low importance scores
  - Results in smaller, more efficient sparse vectors
  
- Other types include `threshold` (keeps tokens above a weight threshold), but `max_ratio` is typically preferred for neural sparse search

#### `prune_ratio`
Specifies the fraction of tokens to retain (as a decimal between 0 and 1):
- **`0.1` (10%)**: Aggressive pruning - keeps top 10% of tokens
  - ✅ Pros: Minimal index size, faster queries, lower memory usage
  - ⚠️ Cons: May lose some semantic information
  
- **`0.2` (20%)**: Moderate pruning - keeps top 20% of tokens
  - Good balance between efficiency and relevance
  
- **`0.3` (30%)**: Conservative pruning - keeps top 30% of tokens
  - ✅ Pros: Better semantic preservation
  - ⚠️ Cons: Larger index, slightly slower queries

#### Example Impact
For a document producing 500 tokens:
- `prune_ratio: 0.1` → keeps ~50 tokens
- `prune_ratio: 0.2` → keeps ~100 tokens  
- `prune_ratio: 0.3` → keeps ~150 tokens

#### Recommendation
Start with `prune_ratio: 0.1` for most production use cases. It provides excellent query performance while maintaining sufficient semantic relevance. Adjust based on your specific relevance vs. performance requirements.

In [42]:
if model_id:
    # Create ingest pipeline for sparse encoding
    pipeline_name = "neural_sparse_ingest_pipeline"
    print(f"🔧 Creating ingest pipeline: {pipeline_name}")
    
    try:
        pipeline_body = {
            "description": "Ingest pipeline for neural sparse search with automatic embedding generation",
            "processors": [
                {
                    "sparse_encoding": {
                        "model_id": model_id,
                        "prune_type": "max_ratio",
                        "prune_ratio": 0.1,
                        "field_map": {
                            "text": "text_embedding"
                        }
                    }
                }
            ]
        }
        
        client.transport.perform_request(
            method='PUT',
            url=f'/_ingest/pipeline/{pipeline_name}',
            body=pipeline_body
        )
        
        print(f"✅ Ingest pipeline created successfully!")
    except Exception as e:
        print(f"❌ Error creating ingest pipeline: {e}")
else:
    print("⚠️ Skipping pipeline creation - model not available")

🔧 Creating ingest pipeline: neural_sparse_ingest_pipeline
✅ Ingest pipeline created successfully!


## Step 4: Create Index with Rank Features

### Overview
The index uses the `rank_features` field type to store sparse embeddings. This allows efficient sparse vector search using an inverted index.

### Understanding `rank_features` Field Type

The `rank_features` field type is specifically designed for storing sparse vectors (token-weight pairs) efficiently. It's the optimal choice for neural sparse search.

#### What are Rank Features?
- **Sparse vectors**: Collections of `{token: weight}` key-value pairs
- **Example**: `{"hello": 6.7, "world": 4.7, "greeting": 0.97}`
- Only non-zero values are stored (extreme sparsity = efficiency)
- Tokens are typically words or subword units (e.g., "##world" for BERT tokenization)
- Weights represent token importance/relevance for that document

#### Why Use `rank_features` Instead of Other Types?

| Aspect | `rank_features` | `dense_vector` | `text` field |
|--------|-----------------|---|---|
| **Storage** | Ultra-efficient (sparse) | Large (dense) | Not designed for vectors |
| **Query Speed** | ⚡ Very fast (inverted index) | 🟡 Medium (approximate) | ❌ Not suitable for embeddings |
| **Memory** | 💾 Low | 📊 Very high | 💾 Medium |
| **Use Case** | Sparse embeddings | Dense embeddings | Full-text search |
| **Scoring** | BM25-like ranking | Similarity scoring | Text matching |

#### Key Characteristics of `rank_features`

1. **Inverted Index Structure**
   - OpenSearch builds an inverted index internally
   - Maps each token → list of documents containing it
   - Enables fast retrieval without scanning all documents
   - Similar to traditional full-text search efficiency

2. **Immutable After Indexing**
   - Cannot be updated in-place
   - Documents must be reindexed to change embeddings
   - Trade-off for query performance gains

3. **No Retrieval of Raw Embeddings**
   - If you set `"excludes": ["text_embedding"]` in `_source` (for disk space savings)
   - The embedding data is permanently discarded
   - Cannot be recovered even from stored data
   - Decision must be made at index creation time

4. **Automatic Tokenization**
   - The sparse encoding model already produces tokens
   - `rank_features` stores them as-is (no additional processing)
   - Each token must be a non-negative number or string identifier

#### Field Mapping Example

```json
"mappings": {
  "properties": {
    "text": {
      "type": "text"              # Original text (full-text searchable)
    },
    "text_embedding": {
      "type": "rank_features"     # Sparse vectors (efficient neural search)
    }
  }
}
```

#### Optimizing Disk Space with `rank_features`

You can exclude embeddings from the `_source` to save 50-60% disk space:

```json
"mappings": {
  "_source": {
    "excludes": ["text_embedding"]  # Don't store embedding in _source
  },
  "properties": {
    "text": { "type": "text" },
    "text_embedding": { "type": "rank_features" }
  }
}
```

**⚠️ Important**: Once excluded from `_source`, embeddings cannot be retrieved. Use this only if you don't need to inspect the embeddings later.

#### Querying with `rank_features`

Neural sparse queries automatically search the `rank_features` field using the inverted index:

```json
{
  "query": {
    "neural_sparse": {
      "text_embedding": {           # Query against rank_features field
        "query_text": "hello world",
        "analyzer": "bert-uncased"
      }
    }
  }
}
```

The search engine:
1. Tokenizes the query text using the specified analyzer
2. Looks up each token in the inverted index
3. Retrieves documents containing those tokens
4. Ranks by combined token weights
5. Returns top matching documents

#### Summary

`rank_features` is the optimal field type for neural sparse search because it:
- ✅ Stores sparse embeddings efficiently
- ✅ Leverages inverted index for fast queries
- ✅ Minimizes memory overhead
- ✅ Provides BM25-style ranking with semantic understanding
- ✅ Scales to billions of documents

In [43]:
if model_id:
    # Create index for neural sparse search
    index_name = "neural_sparse_demo"
    print(f"📇 Creating index: {index_name}")
    
    try:
        # Delete index if it exists
        try:
            client.indices.delete(index=index_name)
            print(f"   Removed existing index")
        except:
            pass
        
        index_body = {
            "settings": {
                "default_pipeline": "neural_sparse_ingest_pipeline",
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "id": {
                        "type": "keyword"
                    },
                    "text": {
                        "type": "text"
                    },
                    "text_embedding": {
                        "type": "rank_features"
                    },
                    "category": {
                        "type": "keyword"
                    },
                    "timestamp": {
                        "type": "date"
                    }
                }
            }
        }
        
        client.indices.create(index=index_name, body=index_body)
        print(f"✅ Index created successfully!")
        print(f"   - Field mapping: text -> text_embedding (rank_features)")
        print(f"   - Default pipeline: neural_sparse_ingest_pipeline")
    except Exception as e:
        print(f"❌ Error creating index: {e}")
else:
    print("⚠️ Skipping index creation - model not available")

📇 Creating index: neural_sparse_demo
✅ Index created successfully!
   - Field mapping: text -> text_embedding (rank_features)
   - Default pipeline: neural_sparse_ingest_pipeline


## Step 5: Use Case 1 - E-Commerce Product Search

### Scenario
Search product catalog efficiently using sparse vectors while maintaining semantic relevance.

In [44]:
if model_id:
    print("\n🛍️ USE CASE 1: E-Commerce Product Search")
    print("="*50)
    
    # Sample e-commerce products (20 stratified across categories)
    products = [
        # Electronics - Audio & Headphones (5 products)
        {
            "id": "prod_001",
            "text": "High-performance wireless Bluetooth headphones with active noise cancellation and 30-hour battery life",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_002",
            "text": "Premium studio-grade over-ear headphones with noise isolation for professional audio engineers",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_003",
            "text": "Compact wireless earbuds with touch controls and dual microphone noise cancellation",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_004",
            "text": "Gaming headset with surround sound 7.1 audio and comfortable memory foam ear cups",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_005",
            "text": "Portable Bluetooth speaker with waterproof design and 360-degree sound output",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        
        # Electronics - Mobile Devices (4 products)
        {
            "id": "prod_006",
            "text": "Ultra-lightweight smartphone with 5G connectivity and exceptional camera system featuring 48MP sensor",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_007",
            "text": "Flagship tablet with large OLED display supporting stylus input for creative professionals",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_008",
            "text": "Rugged smartphone with military-grade durability and exceptional battery endurance in extreme conditions",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_009",
            "text": "Budget-friendly smartphone with excellent processor performance and clean software experience",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        
        # Computer Peripherals (4 products)
        {
            "id": "prod_010",
            "text": "Ergonomic mechanical keyboard with customizable RGB lighting and mechanical Cherry MX switches",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_011",
            "text": "Precision gaming mouse with adjustable DPI settings and programmable side buttons",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_012",
            "text": "Ultra-wide curved monitor with 144Hz refresh rate ideal for immersive gaming and content creation",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_013",
            "text": "Professional external SSD with Thunderbolt connection offering blazing fast data transfer speeds",
            "category": "Electronics",
            "timestamp": datetime.now().isoformat()
        },
        
        # Appliances (3 products)
        {
            "id": "prod_014",
            "text": "Premium stainless steel coffee maker with programmable brewing and thermal carafe temperature control",
            "category": "Appliances",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_015",
            "text": "High-capacity air fryer with smart presets for healthy cooking of various cuisines",
            "category": "Appliances",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_016",
            "text": "Energy-efficient dishwasher with soil sensors and quiet operation under 42 decibels",
            "category": "Appliances",
            "timestamp": datetime.now().isoformat()
        },
        
        # Wearables (3 products)
        {
            "id": "prod_017",
            "text": "Durable waterproof smartwatch with fitness tracking heart rate monitoring and sleep analysis",
            "category": "Wearables",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_018",
            "text": "Advanced fitness band with blood oxygen monitoring and multi-sport tracking capabilities",
            "category": "Wearables",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "prod_019",
            "text": "Smart ring with continuous health metrics tracking and contactless payment support",
            "category": "Wearables",
            "timestamp": datetime.now().isoformat()
        },
        
        # Accessories (1 product)
        {
            "id": "prod_020",
            "text": "Premium carrying case with shock absorption foam and weather-resistant protective design",
            "category": "Accessories",
            "timestamp": datetime.now().isoformat()
        }
    ]
    
    # Ingest products
    print("\n📤 Ingesting 20 products...")
    try:
        for product in products:
            client.index(
                index=index_name,
                id=product['id'],
                body=product
            )
        
        # Refresh index
        client.indices.refresh(index=index_name)
        print(f"✅ Successfully ingested {len(products)} products")
    except Exception as e:
        print(f"❌ Error ingesting products: {e}")


🛍️ USE CASE 1: E-Commerce Product Search

📤 Ingesting 20 products...
✅ Successfully ingested 20 products


In [45]:
if model_id:
    # Search for products
    print("\n🔍 Searching for: 'wireless audio headphones'")
    
    try:
        search_query = {
            "query": {
                "neural_sparse": {
                    "text_embedding": {
                        "query_text": "wireless audio headphones",
                        "analyzer": "bert-uncased"
                    }
                }
            },
            "_source": ["id", "text", "category"],
            "size": 10
        }
        
        response = client.search(index=index_name, body=search_query)
        
        print(f"\n📊 Found {response['hits']['total']['value']} results:")
        print("-" * 80)
        
        for i, hit in enumerate(response['hits']['hits'], 1):
            print(f"\n{i}. Score: {hit['_score']:.4f}")
            print(f"   ID: {hit['_source']['id']}")
            print(f"   Category: {hit['_source']['category']}")
            print(f"   Text: {hit['_source']['text'][:80]}...")
    except Exception as e:
        print(f"❌ Error searching: {e}")


🔍 Searching for: 'wireless audio headphones'

📊 Found 9 results:
--------------------------------------------------------------------------------

1. Score: 9.9664
   ID: prod_001
   Category: Electronics
   Text: High-performance wireless Bluetooth headphones with active noise cancellation an...

2. Score: 9.6712
   ID: prod_002
   Category: Electronics
   Text: Premium studio-grade over-ear headphones with noise isolation for professional a...

3. Score: 8.8853
   ID: prod_004
   Category: Electronics
   Text: Gaming headset with surround sound 7.1 audio and comfortable memory foam ear cup...

4. Score: 8.4226
   ID: prod_003
   Category: Electronics
   Text: Compact wireless earbuds with touch controls and dual microphone noise cancellat...

5. Score: 8.0468
   ID: prod_005
   Category: Electronics
   Text: Portable Bluetooth speaker with waterproof design and 360-degree sound output...

6. Score: 2.2424
   ID: prod_006
   Category: Electronics
   Text: Ultra-lightweight smartphone

## Step 6: Use Case 2 - Customer Support Ticket Search

### Scenario
Find similar support tickets efficiently using neural sparse embeddings to help support agents.

In [46]:
if model_id:
    print("\n🎫 USE CASE 2: Customer Support Ticket Search")
    print("="*50)
    
    # Create separate index for support tickets
    support_index = "neural_sparse_support_tickets"
    
    try:
        # Delete index if it exists
        try:
            client.indices.delete(index=support_index)
        except:
            pass
        
        # Create index
        index_body = {
            "settings": {
                "default_pipeline": "neural_sparse_ingest_pipeline",
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "ticket_id": {"type": "keyword"},
                    "text": {"type": "text"},
                    "text_embedding": {"type": "rank_features"},
                    "priority": {"type": "keyword"},
                    "status": {"type": "keyword"},
                    "resolved_at": {"type": "date"}
                }
            }
        }
        
        client.indices.create(index=support_index, body=index_body)
        print(f"✅ Support tickets index created")
    except Exception as e:
        print(f"❌ Error creating support index: {e}")


🎫 USE CASE 2: Customer Support Ticket Search
✅ Support tickets index created


In [47]:
if model_id:
    # Sample support tickets (20 stratified across categories)
    tickets = [
        # Authentication & Login Issues (5 tickets)
        {
            "ticket_id": "TKT001",
            "text": "Cannot login to my account. Getting 'invalid credentials' error. I have reset password twice.",
            "priority": "high",
            "status": "resolved"
        },
        {
            "ticket_id": "TKT002",
            "text": "Authentication token expired, can't access API. Please refresh my session.",
            "priority": "high",
            "status": "resolved"
        },
        {
            "ticket_id": "TKT003",
            "text": "Two-factor authentication not working. Can't receive verification codes via SMS or email.",
            "priority": "critical",
            "status": "open"
        },
        {
            "ticket_id": "TKT004",
            "text": "Account locked after multiple failed login attempts. Need to unlock immediately.",
            "priority": "high",
            "status": "open"
        },
        {
            "ticket_id": "TKT005",
            "text": "Forgot password link expired before I could reset. Requesting new reset email.",
            "priority": "medium",
            "status": "resolved"
        },
        
        # Payment & Billing Issues (5 tickets)
        {
            "ticket_id": "TKT006",
            "text": "Payment processing is stuck. My order shows pending for 48 hours. Need immediate assistance.",
            "priority": "critical",
            "status": "open"
        },
        {
            "ticket_id": "TKT007",
            "text": "Charged twice for the same purchase. Please refund the duplicate transaction.",
            "priority": "high",
            "status": "resolved"
        },
        {
            "ticket_id": "TKT008",
            "text": "Credit card declined but still charged. Payment gateway showing error message.",
            "priority": "critical",
            "status": "open"
        },
        {
            "ticket_id": "TKT009",
            "text": "Invoice missing from my account. Need receipt for tax documentation.",
            "priority": "medium",
            "status": "resolved"
        },
        {
            "ticket_id": "TKT010",
            "text": "Unable to update payment method. System keeps rejecting new card information.",
            "priority": "high",
            "status": "open"
        },
        
        # Shipping & Delivery Issues (5 tickets)
        {
            "ticket_id": "TKT011",
            "text": "Shipping address verification failed. System won't accept my apartment number.",
            "priority": "medium",
            "status": "resolved"
        },
        {
            "ticket_id": "TKT012",
            "text": "Order tracking number invalid. Can't track my package anywhere.",
            "priority": "medium",
            "status": "open"
        },
        {
            "ticket_id": "TKT013",
            "text": "Package delivered to wrong address. Received neighbor's order instead of mine.",
            "priority": "high",
            "status": "open"
        },
        {
            "ticket_id": "TKT014",
            "text": "Shipping method changed after purchase without my consent. Want refund for expedited shipping.",
            "priority": "medium",
            "status": "resolved"
        },
        {
            "ticket_id": "TKT015",
            "text": "Package damaged upon arrival. Items are broken and unusable.",
            "priority": "high",
            "status": "open"
        },
        
        # Technical & Performance Issues (5 tickets)
        {
            "ticket_id": "TKT016",
            "text": "Website very slow, pages taking 30 seconds to load. Tried different browsers.",
            "priority": "high",
            "status": "open"
        },
        {
            "ticket_id": "TKT017",
            "text": "Mobile app crashes when accessing product catalog. Error on startup.",
            "priority": "high",
            "status": "resolved"
        },
        {
            "ticket_id": "TKT018",
            "text": "Search functionality not working. Results page shows error and won't load.",
            "priority": "critical",
            "status": "open"
        },
        {
            "ticket_id": "TKT019",
            "text": "Checkout page stuck on payment processing screen. Unable to complete purchase.",
            "priority": "critical",
            "status": "open"
        },
        {
            "ticket_id": "TKT020",
            "text": "Session timeout too quick. Getting logged out while browsing products.",
            "priority": "medium",
            "status": "resolved"
        }
    ]
    
    # Ingest tickets
    print("\n📤 Ingesting 20 support tickets...")
    try:
        for ticket in tickets:
            client.index(
                index=support_index,
                id=ticket['ticket_id'],
                body=ticket
            )
        
        client.indices.refresh(index=support_index)
        print(f"✅ Successfully ingested {len(tickets)} support tickets")
    except Exception as e:
        print(f"❌ Error ingesting tickets: {e}")


📤 Ingesting 20 support tickets...
✅ Successfully ingested 20 support tickets


In [48]:
if model_id:
    # Search support tickets
    print("\n🔍 New Ticket: 'Account login authentication failed'")
    print("   Finding similar tickets...\n")
    
    try:
        search_query = {
            "query": {
                "neural_sparse": {
                    "text_embedding": {
                        "query_text": "login authentication failed",
                        "analyzer": "bert-uncased"
                    }
                }
            },
            "_source": ["ticket_id", "text", "priority", "status"],
            "size": 5
        }
        
        response = client.search(index=support_index, body=search_query)
        
        print(f"📊 Found {response['hits']['total']['value']} similar tickets:")
        print("-" * 80)
        
        for i, hit in enumerate(response['hits']['hits'], 1):
            print(f"\n{i}. Relevance Score: {hit['_score']:.4f}")
            print(f"   Ticket ID: {hit['_source']['ticket_id']}")
            print(f"   Priority: {hit['_source']['priority']} | Status: {hit['_source']['status']}")
            print(f"   Issue: {hit['_source']['text'][:75]}...")
    except Exception as e:
        print(f"❌ Error searching support tickets: {e}")


🔍 New Ticket: 'Account login authentication failed'
   Finding similar tickets...

📊 Found 12 similar tickets:
--------------------------------------------------------------------------------

1. Relevance Score: 8.9597
   Ticket ID: TKT004
   Priority: high | Status: open
   Issue: Account locked after multiple failed login attempts. Need to unlock immedia...

2. Relevance Score: 7.8385
   Ticket ID: TKT001
   Priority: high | Status: resolved
   Issue: Cannot login to my account. Getting 'invalid credentials' error. I have res...

3. Relevance Score: 5.9660
   Ticket ID: TKT011
   Priority: medium | Status: resolved
   Issue: Shipping address verification failed. System won't accept my apartment numb...

4. Relevance Score: 5.3457
   Ticket ID: TKT002
   Priority: high | Status: resolved
   Issue: Authentication token expired, can't access API. Please refresh my session....

5. Relevance Score: 4.5373
   Ticket ID: TKT003
   Priority: critical | Status: open
   Issue: Two-factor aut

## Step 7: Use Case 3 - Document Repository Search

### Scenario
Search large document repositories (technical docs, research papers, etc.) with semantic relevance.

In [49]:
if model_id:
    print("\n📚 USE CASE 3: Document Repository Search")
    print("="*50)
    
    # Create index for documents
    docs_index = "neural_sparse_documents"
    
    try:
        # Delete index if it exists
        try:
            client.indices.delete(index=docs_index)
        except:
            pass
        
        # Create index
        index_body = {
            "settings": {
                "default_pipeline": "neural_sparse_ingest_pipeline",
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "doc_id": {"type": "keyword"},
                    "text": {"type": "text"},
                    "text_embedding": {"type": "rank_features"},
                    "doc_type": {"type": "keyword"},
                    "author": {"type": "keyword"},
                    "created_date": {"type": "date"}
                }
            }
        }
        
        client.indices.create(index=docs_index, body=index_body)
        print(f"✅ Documents index created")
    except Exception as e:
        print(f"❌ Error creating documents index: {e}")


📚 USE CASE 3: Document Repository Search
✅ Documents index created


In [50]:
if model_id:
    # Sample documents (20 stratified across technical topics)
    documents = [
        # Machine Learning Fundamentals (4 documents)
        {
            "doc_id": "DOC001",
            "text": "Machine learning models require careful preprocessing of input data including normalization, tokenization, and feature scaling to achieve optimal performance.",
            "doc_type": "technical_guide",
            "author": "Dr. Smith"
        },
        {
            "doc_id": "DOC002",
            "text": "Supervised learning algorithms learn from labeled training data to predict outcomes. Classification and regression are two primary supervised learning tasks.",
            "doc_type": "technical_guide",
            "author": "Dr. Smith"
        },
        {
            "doc_id": "DOC003",
            "text": "Unsupervised learning discovers hidden patterns in unlabeled data through clustering and dimensionality reduction techniques for exploratory data analysis.",
            "doc_type": "research_paper",
            "author": "Prof. Anderson"
        },
        {
            "doc_id": "DOC004",
            "text": "Model evaluation metrics like precision, recall, and F1-score help assess classifier performance and guide hyperparameter optimization decisions.",
            "doc_type": "technical_guide",
            "author": "Dr. Wilson"
        },
        
        # Deep Learning & Neural Networks (4 documents)
        {
            "doc_id": "DOC005",
            "text": "Neural networks consist of interconnected layers of neurons that process information through non-linear activation functions enabling complex pattern recognition.",
            "doc_type": "research_paper",
            "author": "Prof. Johnson"
        },
        {
            "doc_id": "DOC006",
            "text": "Convolutional neural networks excel at image processing tasks by using convolutional filters to extract spatial features from visual data efficiently.",
            "doc_type": "research_paper",
            "author": "Prof. Garcia"
        },
        {
            "doc_id": "DOC007",
            "text": "Recurrent neural networks process sequential data through hidden state mechanisms enabling them to capture temporal dependencies in time series.",
            "doc_type": "technical_guide",
            "author": "Dr. Martinez"
        },
        {
            "doc_id": "DOC008",
            "text": "Transformer architectures revolutionized NLP by replacing recurrence with self-attention mechanisms achieving state-of-the-art performance on language tasks.",
            "doc_type": "research_paper",
            "author": "Prof. Lee"
        },
        
        # Natural Language Processing (4 documents)
        {
            "doc_id": "DOC009",
            "text": "Natural language processing leverages transformer models with attention mechanisms to understand contextual relationships between words in sequential text data.",
            "doc_type": "research_paper",
            "author": "Dr. Lee"
        },
        {
            "doc_id": "DOC010",
            "text": "Word embeddings like Word2Vec and GloVe represent words as dense vectors capturing semantic relationships enabling effective text analysis and similarity computation.",
            "doc_type": "technical_guide",
            "author": "Prof. Chen"
        },
        {
            "doc_id": "DOC011",
            "text": "Named entity recognition identifies and classifies named entities like persons, organizations, and locations from unstructured text documents.",
            "doc_type": "technical_guide",
            "author": "Dr. Kumar"
        },
        {
            "doc_id": "DOC012",
            "text": "Sentiment analysis determines emotional tone and opinion polarity in text using lexicon-based and machine learning approaches for customer feedback analysis.",
            "doc_type": "research_paper",
            "author": "Prof. Patel"
        },
        
        # Distributed Systems & Infrastructure (4 documents)
        {
            "doc_id": "DOC013",
            "text": "Distributed computing systems utilize clustering algorithms and consensus protocols to maintain data consistency across multiple nodes in fault-tolerant architectures.",
            "doc_type": "technical_guide",
            "author": "Dr. Brown"
        },
        {
            "doc_id": "DOC014",
            "text": "MapReduce framework enables processing large datasets across distributed clusters by dividing work into map and reduce phases for parallel computation.",
            "doc_type": "research_paper",
            "author": "Prof. Davis"
        },
        {
            "doc_id": "DOC015",
            "text": "NoSQL databases provide scalable storage for unstructured data using document models, key-value stores, and column families replacing traditional relational databases.",
            "doc_type": "technical_guide",
            "author": "Dr. Taylor"
        },
        {
            "doc_id": "DOC016",
            "text": "Microservices architecture decomposes applications into independently deployable services improving scalability and enabling continuous deployment practices.",
            "doc_type": "technical_guide",
            "author": "Prof. Harris"
        },
        
        # Search & Information Retrieval (4 documents)
        {
            "doc_id": "DOC017",
            "text": "Search engines employ inverted indexing and ranking algorithms with query expansion techniques to retrieve relevant documents from large-scale text collections efficiently.",
            "doc_type": "technical_guide",
            "author": "Prof. Chen"
        },
        {
            "doc_id": "DOC018",
            "text": "Information retrieval systems combine keyword matching with semantic understanding using vector embeddings for improved ranking and relevance assessment.",
            "doc_type": "research_paper",
            "author": "Prof. Robinson"
        },
        {
            "doc_id": "DOC019",
            "text": "BM25 ranking function balances term frequency and document length normalization for robust full-text search performance in document retrieval systems.",
            "doc_type": "technical_guide",
            "author": "Dr. Clark"
        },
        {
            "doc_id": "DOC020",
            "text": "Semantic search using sparse neural embeddings combines efficiency with relevance by identifying token-weight relationships for improved retrieval accuracy.",
            "doc_type": "research_paper",
            "author": "Prof. White"
        }
    ]
    
    # Ingest documents
    print("\n📤 Ingesting 20 technical documents...")
    try:
        for doc in documents:
            client.index(
                index=docs_index,
                id=doc['doc_id'],
                body=doc
            )
        
        client.indices.refresh(index=docs_index)
        print(f"✅ Successfully ingested {len(documents)} documents")
    except Exception as e:
        print(f"❌ Error ingesting documents: {e}")


📤 Ingesting 20 technical documents...
✅ Successfully ingested 20 documents


In [51]:
if model_id:
    # Search documents
    print("\n🔍 Searching for: 'deep learning neural networks training'\n")
    
    try:
        search_query = {
            "query": {
                "neural_sparse": {
                    "text_embedding": {
                        "query_text": "deep learning neural networks training",
                        "analyzer": "bert-uncased"
                    }
                }
            },
            "_source": ["doc_id", "text", "doc_type", "author"],
            "size": 10
        }
        
        response = client.search(index=docs_index, body=search_query)
        
        print(f"📊 Found {response['hits']['total']['value']} relevant documents:")
        print("-" * 80)
        
        for i, hit in enumerate(response['hits']['hits'], 1):
            print(f"\n{i}. Relevance Score: {hit['_score']:.4f}")
            print(f"   Doc ID: {hit['_source']['doc_id']}")
            print(f"   Type: {hit['_source']['doc_type']} | Author: {hit['_source']['author']}")
            print(f"   Summary: {hit['_source']['text'][:80]}...")
    except Exception as e:
        print(f"❌ Error searching documents: {e}")


🔍 Searching for: 'deep learning neural networks training'

📊 Found 11 relevant documents:
--------------------------------------------------------------------------------

1. Relevance Score: 9.9394
   Doc ID: DOC005
   Type: research_paper | Author: Prof. Johnson
   Summary: Neural networks consist of interconnected layers of neurons that process informa...

2. Relevance Score: 8.4649
   Doc ID: DOC007
   Type: technical_guide | Author: Dr. Martinez
   Summary: Recurrent neural networks process sequential data through hidden state mechanism...

3. Relevance Score: 7.4427
   Doc ID: DOC006
   Type: research_paper | Author: Prof. Garcia
   Summary: Convolutional neural networks excel at image processing tasks by using convoluti...

4. Relevance Score: 7.2702
   Doc ID: DOC001
   Type: technical_guide | Author: Dr. Smith
   Summary: Machine learning models require careful preprocessing of input data including no...

5. Relevance Score: 7.1659
   Doc ID: DOC002
   Type: technical_guide |

## Step 8: Performance Comparison & Analysis

### Key Metrics
Compare neural sparse search with traditional BM25 and dense semantic search.

In [52]:
if model_id:
    print("\n📊 PERFORMANCE COMPARISON ANALYSIS")
    print("="*80)
    
    comparison_data = {
        "Method": ["BM25 (Traditional Lexical)", "Dense Semantic Search", "Neural Sparse Search"],
        "Indexing Speed": ["Very Fast ⚡", "Slow 🐢", "Fast ⚡⚡"],
        "Query Speed": ["Very Fast ⚡", "Medium 🟡", "Very Fast ⚡⚡"],
        "Memory Usage": ["Low 💾", "Very High 📊📊", "Low-Medium 💾"],
        "Relevance": ["Medium 🟡", "Excellent ✅", "Excellent ✅"],
        "Semantic Understanding": ["None ❌", "Excellent ✅", "Excellent ✅"],
        "Scalability": ["Excellent ✅", "Limited ⚠️", "Excellent ✅"]
    }
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    print("\n✨ Key Advantages of Neural Sparse Search:")
    print("  1. Combines efficiency of sparse retrieval with semantic relevance")
    print("  2. Low memory footprint compared to dense embeddings")
    print("  3. Fast indexing and query performance at scale")
    print("  4. Interpretable token-weight pairs provide explainability")
    print("  5. Hybrid approach balances performance and accuracy")


📊 PERFORMANCE COMPARISON ANALYSIS
                    Method Indexing Speed  Query Speed Memory Usage   Relevance Semantic Understanding Scalability
BM25 (Traditional Lexical)    Very Fast ⚡  Very Fast ⚡        Low 💾    Medium 🟡                 None ❌ Excellent ✅
     Dense Semantic Search         Slow 🐢     Medium 🟡 Very High 📊📊 Excellent ✅            Excellent ✅  Limited ⚠️
      Neural Sparse Search        Fast ⚡⚡ Very Fast ⚡⚡ Low-Medium 💾 Excellent ✅            Excellent ✅ Excellent ✅

✨ Key Advantages of Neural Sparse Search:
  1. Combines efficiency of sparse retrieval with semantic relevance
  2. Low memory footprint compared to dense embeddings
  3. Fast indexing and query performance at scale
  4. Interpretable token-weight pairs provide explainability
  5. Hybrid approach balances performance and accuracy


## Step 9: Advanced Configuration - ANN Search Mode

### Overview
Neural Sparse ANN (Approximate Nearest Neighbor) search provides optimized query performance for large-scale datasets using clustering and approximate search techniques.

In [53]:
print("\n⚡ NEURAL SPARSE ANN SEARCH MODE")
print("="*80)
print("\n🎯 When to use Neural Sparse ANN Search:")
print("  • Datasets with millions to billions of documents")
print("  • High-throughput query scenarios requiring sub-100ms response times")
print("  • Need >90% recall with minimal latency trade-off")

print("\n📋 Configuration for ANN Search:")
print("""
{
  "settings": {
    "index.sparse": true
  },
  "mappings": {
    "properties": {
      "sparse_embedding": {
        "type": "sparse_vector",
        "method": {
          "name": "seismic",
          "parameters": {
            "n_postings": 4000,          # Top docs per token
            "cluster_ratio": 0.1,        # Clustering granularity
            "summary_prune_ratio": 0.4,  # Summary vector pruning
            "approximate_threshold": 1000000  # ANN activation threshold
          }
        }
      }
    }
  }
}
""")

print("\n🔧 Query Parameters for ANN Search:")
query_params = {
    "Parameter": ["k", "top_n", "heap_factor"],
    "Purpose": ["Number of top results to return", "Top query tokens to retain", "Recall vs performance trade-off"],
    "Typical Value": ["10-100", "10-50", "1.0-2.0"]
}
params_df = pd.DataFrame(query_params)
print(params_df.to_string(index=False))


⚡ NEURAL SPARSE ANN SEARCH MODE

🎯 When to use Neural Sparse ANN Search:
  • Datasets with millions to billions of documents
  • High-throughput query scenarios requiring sub-100ms response times
  • Need >90% recall with minimal latency trade-off

📋 Configuration for ANN Search:

{
  "settings": {
    "index.sparse": true
  },
  "mappings": {
    "properties": {
      "sparse_embedding": {
        "type": "sparse_vector",
        "method": {
          "name": "seismic",
          "parameters": {
            "n_postings": 4000,          # Top docs per token
            "cluster_ratio": 0.1,        # Clustering granularity
            "summary_prune_ratio": 0.4,  # Summary vector pruning
            "approximate_threshold": 1000000  # ANN activation threshold
          }
        }
      }
    }
  }
}


🔧 Query Parameters for ANN Search:
  Parameter                         Purpose Typical Value
          k Number of top results to return        10-100
      top_n      Top query tokens t

## Create ANN index

In [54]:
if model_id:
    # Create index for neural sparse search
    index_name = "neural_sparse_demo_ann"
    print(f"📇 Creating index: {index_name}")
    
    try:
        # Delete index if it exists
        try:
            client.indices.delete(index=index_name)
            print(f"   Removed existing index")
        except:
            pass
        
        index_body = {
            "settings": {
                "default_pipeline": "neural_sparse_ingest_pipeline",
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "index.sparse": True
            },
            "mappings": {
                "properties": {
                    "id": {
                        "type": "keyword"
                    },
                    "text": {
                        "type": "text"
                    },
                          "sparse_embedding": {
        "type": "sparse_vector",
        "method": {
          "name": "seismic",
          "parameters": {
            "n_postings": 4000,          # Top docs per token
            "cluster_ratio": 0.1,        # Clustering granularity
            "summary_prune_ratio": 0.4,  # Summary vector pruning
            "approximate_threshold": 1000000  # ANN activation threshold
          }
        }
      },
                    "category": {
                        "type": "keyword"
                    },
                    "timestamp": {
                        "type": "date"
                    }
                }
            }
        }
        
        client.indices.create(index=index_name, body=index_body)
        print(f"✅ Index created successfully!")
        print(f"   - Field mapping: text -> text_embedding (rank_features)")
        print(f"   - Default pipeline: neural_sparse_ingest_pipeline")
    except Exception as e:
        print(f"❌ Error creating index: {e}")
else:
    print("⚠️ Skipping index creation - model not available")

📇 Creating index: neural_sparse_demo_ann
✅ Index created successfully!
   - Field mapping: text -> text_embedding (rank_features)
   - Default pipeline: neural_sparse_ingest_pipeline


## Ingest ANN sparse documents

In [55]:
if model_id:
    print("\n📤 INGESTING DOCUMENTS INTO NEURAL SPARSE ANN INDEX")
    print("="*80)
    
    # Note: The index has a field mapping issue - the pipeline maps "text" -> "text_embedding"
    # but the ANN index expects "sparse_embedding". We need to either:
    # 1. Update the pipeline to map to "sparse_embedding", or
    # 2. Manually provide sparse embeddings, or
    # 3. Create a new pipeline for this index
    
    # For this demo, let's create a new pipeline specific to this ANN index
    ann_pipeline_name = "neural_sparse_ann_ingest_pipeline"
    print(f"\n🔧 Creating ANN-specific ingest pipeline: {ann_pipeline_name}")
    
    try:
        ann_pipeline_body = {
            "description": "Ingest pipeline for neural sparse ANN search with sparse_embedding field",
            "processors": [
                {
                    "sparse_encoding": {
                        "model_id": model_id,
                        "prune_type": "max_ratio",
                        "prune_ratio": 0.1,
                        "field_map": {
                            "text": "sparse_embedding"  # Map to sparse_embedding instead of text_embedding
                        }
                    }
                }
            ]
        }
        
        client.transport.perform_request(
            method='PUT',
            url=f'/_ingest/pipeline/{ann_pipeline_name}',
            body=ann_pipeline_body
        )
        
        print(f"✅ ANN ingest pipeline created successfully!")
        print(f"   Field mapping: text -> sparse_embedding")
    except Exception as e:
        print(f"❌ Error creating ANN pipeline: {e}")
    
    # Update the index settings to use the new pipeline
    try:
        client.indices.put_settings(
            index=index_name,
            body={"index": {"default_pipeline": ann_pipeline_name}}
        )
        print(f"✅ Index '{index_name}' updated to use ANN pipeline")
    except Exception as e:
        print(f"⚠️ Could not update index settings: {e}")
    
    # Sample documents for Neural Sparse ANN (larger dataset for ANN benefits)
    print("\n📝 Preparing 30 documents across diverse topics...")
    
    ann_documents = [
        # Technology & AI (8 documents)
        {
            "id": "ann_001",
            "text": "Artificial intelligence systems using deep learning neural networks demonstrate remarkable performance in computer vision tasks including image classification and object detection",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_002",
            "text": "Natural language processing models leverage transformer architecture with attention mechanisms to understand semantic context and generate human-like text responses",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_003",
            "text": "Machine learning algorithms require extensive training data preprocessing including normalization feature engineering and hyperparameter tuning for optimal model performance",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_004",
            "text": "Cloud computing platforms provide scalable infrastructure services including virtual machines containerization and serverless computing for distributed applications",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_005",
            "text": "Blockchain technology enables decentralized distributed ledger systems with cryptographic security ensuring transparent immutable transaction records",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_006",
            "text": "Quantum computing leverages quantum mechanics principles including superposition and entanglement for exponentially faster computation on specific problem domains",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_007",
            "text": "Edge computing brings data processing closer to IoT devices reducing latency and bandwidth consumption for real-time applications in smart cities",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_008",
            "text": "Cybersecurity frameworks implement defense mechanisms including encryption firewalls intrusion detection systems and zero-trust architecture for enterprise protection",
            "category": "Technology",
            "timestamp": datetime.now().isoformat()
        },
        
        # Healthcare & Medicine (7 documents)
        {
            "id": "ann_009",
            "text": "Medical diagnostic imaging systems utilize advanced computational algorithms for disease detection in radiology including MRI CT scans and ultrasound analysis",
            "category": "Healthcare",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_010",
            "text": "Genomic sequencing technologies enable personalized medicine approaches by analyzing DNA variations for targeted therapeutic interventions and drug development",
            "category": "Healthcare",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_011",
            "text": "Telemedicine platforms facilitate remote patient consultations using video conferencing and digital health monitoring devices for accessible healthcare delivery",
            "category": "Healthcare",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_012",
            "text": "Pharmaceutical research employs high-throughput screening and computational drug design to discover novel therapeutic compounds for disease treatment",
            "category": "Healthcare",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_013",
            "text": "Surgical robotics enhance precision in minimally invasive procedures using computer-assisted navigation and haptic feedback for improved patient outcomes",
            "category": "Healthcare",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_014",
            "text": "Electronic health records integrate patient medical history laboratory results and treatment plans for coordinated care across healthcare providers",
            "category": "Healthcare",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_015",
            "text": "Immunotherapy treatments harness the immune system to target cancer cells using checkpoint inhibitors and CAR T-cell therapy for oncology patients",
            "category": "Healthcare",
            "timestamp": datetime.now().isoformat()
        },
        
        # Finance & Economics (7 documents)
        {
            "id": "ann_016",
            "text": "Algorithmic trading systems execute high-frequency trades using quantitative models and real-time market data analysis for optimized investment strategies",
            "category": "Finance",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_017",
            "text": "Cryptocurrency markets employ blockchain-based digital assets with decentralized exchange platforms for peer-to-peer financial transactions",
            "category": "Finance",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_018",
            "text": "Risk management frameworks utilize statistical modeling and stress testing to assess portfolio vulnerability and regulatory compliance in banking institutions",
            "category": "Finance",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_019",
            "text": "Financial technology innovations include mobile payment solutions digital wallets and robo-advisors for automated investment portfolio management",
            "category": "Finance",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_020",
            "text": "Credit scoring algorithms analyze consumer financial behavior using machine learning for loan approval decisions and interest rate determination",
            "category": "Finance",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_021",
            "text": "Macroeconomic indicators including GDP inflation rates and employment statistics guide monetary policy decisions by central banking authorities",
            "category": "Finance",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_022",
            "text": "Derivatives markets trade complex financial instruments including options futures and swaps for hedging strategies and speculative investment positions",
            "category": "Finance",
            "timestamp": datetime.now().isoformat()
        },
        
        # Environmental Science (8 documents)
        {
            "id": "ann_023",
            "text": "Climate modeling systems simulate atmospheric dynamics and ocean currents using supercomputers to predict global temperature changes and weather patterns",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_024",
            "text": "Renewable energy technologies harness solar photovoltaic panels wind turbines and hydroelectric systems for sustainable electricity generation",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_025",
            "text": "Environmental monitoring networks deploy sensor arrays to measure air quality water contamination and ecosystem biodiversity for conservation efforts",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_026",
            "text": "Carbon capture and storage technologies mitigate greenhouse gas emissions by capturing CO2 from industrial sources for underground sequestration",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_027",
            "text": "Precision agriculture employs GPS-guided machinery and satellite imagery for optimized crop management reducing pesticide use and water consumption",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_028",
            "text": "Ocean acidification studies monitor pH changes in marine ecosystems examining impacts on coral reefs and shellfish populations from carbon dioxide absorption",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_029",
            "text": "Waste management systems integrate recycling programs composting facilities and waste-to-energy conversion for circular economy sustainability goals",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        },
        {
            "id": "ann_030",
            "text": "Biodiversity conservation strategies protect endangered species through habitat restoration wildlife corridors and anti-poaching enforcement measures",
            "category": "Environment",
            "timestamp": datetime.now().isoformat()
        }
    ]
    
    # Ingest documents using the ANN pipeline
    print(f"\n📤 Ingesting {len(ann_documents)} documents...")
    print(f"   Using pipeline: {ann_pipeline_name}")
    
    try:
        success_count = 0
        for doc in ann_documents:
            try:
                client.index(
                    index=index_name,
                    id=doc['id'],
                    body=doc,
                    pipeline=ann_pipeline_name  # Explicitly use the ANN pipeline
                )
                success_count += 1
            except Exception as doc_error:
                print(f"   ⚠️ Failed to index {doc['id']}: {doc_error}")
        
        # Refresh index to make documents searchable
        client.indices.refresh(index=index_name)
        
        print(f"\n✅ Successfully ingested {success_count}/{len(ann_documents)} documents")
        print(f"   • 8 Technology documents")
        print(f"   • 7 Healthcare documents")
        print(f"   • 7 Finance documents")
        print(f"   • 8 Environment documents")
        
        # Verify ingestion
        count = client.count(index=index_name)
        print(f"\n📊 Total documents in index: {count['count']}")
        
    except Exception as e:
        print(f"❌ Error during bulk ingestion: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠️ Skipping document ingestion - model not available")


📤 INGESTING DOCUMENTS INTO NEURAL SPARSE ANN INDEX

🔧 Creating ANN-specific ingest pipeline: neural_sparse_ann_ingest_pipeline
✅ ANN ingest pipeline created successfully!
   Field mapping: text -> sparse_embedding
✅ Index 'neural_sparse_demo_ann' updated to use ANN pipeline

📝 Preparing 30 documents across diverse topics...

📤 Ingesting 30 documents...
   Using pipeline: neural_sparse_ann_ingest_pipeline

✅ Successfully ingested 30/30 documents
   • 8 Technology documents
   • 7 Healthcare documents
   • 7 Finance documents
   • 8 Environment documents

📊 Total documents in index: 30


## ANN Search

In [56]:
if model_id:
    print("\n🔍 NEURAL SPARSE ANN SEARCH QUERY EXAMPLE")
    print("="*80)
    
    # Example ANN search query using official documentation format
    print("\nSearching documents using Neural Sparse ANN mode...")
    print("Query: 'machine learning artificial intelligence deep learning'")
    
    try:
        ann_search_query = {
            "query": {
                "neural_sparse": {
                    "sparse_embedding": {
                        "query_text": "machine learning artificial intelligence deep learning",
                        "model_id": model_id,
                        "method_parameters": {
                            "k": 5,          # Number of top results to return
                            "top_n": 10,     # Top query tokens to retain
                            "heap_factor": 1.0  # Recall vs performance trade-off
                        }
                    }
                }
            },
            "_source": ["id", "text", "category"]
        }
        
        response = client.search(index=index_name, body=ann_search_query)
        
        print(f"\n📊 ANN Search Results ({response['hits']['total']['value']} matches):")
        print("-" * 80)
        
        for i, hit in enumerate(response['hits']['hits'], 1):
            print(f"\n{i}. Relevance Score: {hit['_score']:.4f}")
            print(f"   Document ID: {hit['_source']['id']}")
            print(f"   Category: {hit['_source']['category']}")
            print(f"   Content: {hit['_source']['text'][:80]}...")
        
        print("\n💡 Key ANN Query Features Used:")
        print("   • method_parameters.k=5 - Returns top 5 documents")
        print("   • method_parameters.top_n=10 - Uses top 10 query tokens")
        print("   • method_parameters.heap_factor=1.0 - Exact recall mode")
        print("   • Query text automatically tokenized by sparse encoding model")
        print("   • SEISMIC algorithm for clustered inverted index search")
        
    except Exception as e:
        print(f"❌ Error executing ANN search: {e}")
else:
    print("⚠️ Skipping ANN search example - model not available")


🔍 NEURAL SPARSE ANN SEARCH QUERY EXAMPLE

Searching documents using Neural Sparse ANN mode...
Query: 'machine learning artificial intelligence deep learning'

📊 ANN Search Results (28 matches):
--------------------------------------------------------------------------------

1. Relevance Score: 4.5185
   Document ID: ann_001
   Category: Technology
   Content: Artificial intelligence systems using deep learning neural networks demonstrate ...

2. Relevance Score: 2.1186
   Document ID: ann_003
   Category: Technology
   Content: Machine learning algorithms require extensive training data preprocessing includ...

3. Relevance Score: 1.0815
   Document ID: ann_006
   Category: Technology
   Content: Quantum computing leverages quantum mechanics principles including superposition...

4. Relevance Score: 1.0130
   Document ID: ann_020
   Category: Finance
   Content: Credit scoring algorithms analyze consumer financial behavior using machine lear...

5. Relevance Score: 0.8795
   Document

In [58]:
if model_id:
    print("\n🎯 ANN SEARCH QUERY EXAMPLES WITH DIFFERENT PARAMETERS")
    print("="*80)
    
    # Example 1: Using k parameter (number of results)
    print("\n📌 Example 1: Varying 'k' - Number of Results to Return")
    print("-" * 80)
    
    for k_value in [3, 5, 10]:
        print(f"\n🔍 Query with k={k_value} (return top {k_value} results):")
        try:
            query_k = {
                "size": k_value,  # limit returned hits; default is 10 if omitted
                "query": {
                    "neural_sparse": {
                        "sparse_embedding": {
                            "query_text": "healthcare medical diagnostic imaging",
                            "model_id": model_id,
                            "method_parameters": {
                                "k": k_value,
                                "top_n": 8,
                                "heap_factor": 1.0
                            }
                        }
                    }
                },
                "_source": ["id", "text", "category"]
            }
            
            response = client.search(index=index_name, body=query_k)
            print(f"   Results returned: {len(response['hits']['hits'])}")
            for i, hit in enumerate(response['hits']['hits'], 1):
                print(f"      {i}. {hit['_source']['id']} (score: {hit['_score']:.4f})")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    # Example 2: Using top_n parameter (query tokens)
    print("\n\n📌 Example 2: Varying 'top_n' - Number of Query Tokens to Retain")
    print("-" * 80)
    print("   top_n controls how many highest-weighted query tokens are used for search")
    
    query_text = "blockchain cryptocurrency decentralized financial technology systems"
    
    for top_n_value in [5, 10, 15]:
        print(f"\n🔍 Query with top_n={top_n_value} tokens:")
        try:
            query_topn = {
                "query": {
                    "neural_sparse": {
                        "sparse_embedding": {
                            "query_text": query_text,
                            "model_id": model_id,
                            "method_parameters": {
                                "k": 3,
                                "top_n": top_n_value,
                                "heap_factor": 1.0
                            }
                        }
                    }
                },
                "_source": ["id", "category"]
            }
            
            response = client.search(index=index_name, body=query_topn)
            print(f"   Matched {response['hits']['total']['value']} documents:")
            for i, hit in enumerate(response['hits']['hits'], 1):
                print(f"      {i}. {hit['_source']['id']} - {hit['_source']['category']} (score: {hit['_score']:.4f})")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    
    # Example 3: Using heap_factor for recall/performance trade-off
    print("\n\n📌 Example 3: Varying 'heap_factor' - Recall vs Performance Trade-off")
    print("-" * 80)
    print("   heap_factor: Controls the balance between search quality and speed")
    print("   • 1.0 = Exact recall, slower (100% accuracy)")
    print("   • 1.2 = High recall, slightly faster (98-99% accuracy)")
    print("   • 1.5 = Good recall, balanced (95-98% accuracy)")
    print("   • 2.0 = Acceptable recall, much faster (90-95% accuracy)")
    
    print("\n🔍 Same Query with Different heap_factor Values:")
    query_text = "renewable energy solar wind hydroelectric power generation"
    
    for heap_factor_value in [1.0, 1.2, 1.5, 2.0]:
        print(f"\n   heap_factor: {heap_factor_value}")
        try:
            query_heap = {
                "query": {
                    "neural_sparse": {
                        "sparse_embedding": {
                            "query_text": query_text,
                            "model_id": model_id,
                            "method_parameters": {
                                "k": 5,
                                "top_n": 10,
                                "heap_factor": heap_factor_value
                            }
                        }
                    }
                },
                "_source": ["id", "text"]
            }
            
            response = client.search(index=index_name, body=query_heap)
            print(f"      Total hits: {response['hits']['total']['value']}")
            for i, hit in enumerate(response['hits']['hits'][:3], 1):
                print(f"      {i}. {hit['_source']['id']} (score: {hit['_score']:.4f})")
        except Exception as e:
            print(f"      ❌ Error: {e}")
    
    # Example 4: Combined parameters with filtering
    print("\n\n📌 Example 4: Production Query with Filtering")
    print("-" * 80)
    
    print("\n🔍 Production Query Configuration:")
    print("   - k=10 (return top 10 results)")
    print("   - top_n=15 (use top 15 query tokens)")
    print("   - heap_factor=1.2 (balanced recall/speed)")
    print("   - Category filter for 'Technology' documents")
    
    try:
        production_query = {
            "query": {
                "bool": {
                    "must": [
                        {
                            "neural_sparse": {
                                "sparse_embedding": {
                                    "query_text": "artificial intelligence neural networks deep learning models",
                                    "model_id": model_id,
                                    "method_parameters": {
                                        "k": 10,
                                        "top_n": 15,
                                        "heap_factor": 1.2
                                    }
                                }
                            }
                        }
                    ],
                    "filter": [
                        {"term": {"category": "Technology"}}
                    ]
                }
            },
            "_source": ["id", "text", "category"]
        }
        
        response = client.search(index=index_name, body=production_query)
        print(f"\n   ✅ Query Results: {len(response['hits']['hits'])} documents returned")
        print(f"   Total matches: {response['hits']['total']['value']}")
        print(f"\n   Top Results:")
        for i, hit in enumerate(response['hits']['hits'][:5], 1):
            print(f"      {i}. {hit['_source']['id']} - Score: {hit['_score']:.4f}")
            print(f"         {hit['_source']['text'][:70]}...")
    except Exception as e:
        print(f"   ❌ Error: {e}")
    
    print("\n\n💡 Parameter Guidelines (per OpenSearch docs):")
    print("   • k: Number of top results to return (typically 5-100)")
    print("   • top_n: Number of query tokens to retain (typically 5-50)")
    print("   • heap_factor: Controls recall vs speed trade-off")
    print("     - 1.0: Exact recall (slowest, highest accuracy)")
    print("     - 1.2-1.5: Balanced (recommended for production)")
    print("     - 2.0+: Fast approximate search (lower recall)")
    print("   • Use filters for pre-filtering or post-filtering scenarios")

else:
    print("⚠️ Skipping parameter examples - model not available")



🎯 ANN SEARCH QUERY EXAMPLES WITH DIFFERENT PARAMETERS

📌 Example 1: Varying 'k' - Number of Results to Return
--------------------------------------------------------------------------------

🔍 Query with k=3 (return top 3 results):
   Results returned: 3
      1. ann_009 (score: 4.5111)
      2. ann_014 (score: 3.4370)
      3. ann_011 (score: 2.4365)

🔍 Query with k=5 (return top 5 results):
   Results returned: 5
      1. ann_009 (score: 4.5111)
      2. ann_014 (score: 3.4370)
      3. ann_011 (score: 2.4365)
      4. ann_012 (score: 1.1346)
      5. ann_010 (score: 0.9982)

🔍 Query with k=10 (return top 10 results):
   Results returned: 10
      1. ann_009 (score: 4.5111)
      2. ann_014 (score: 3.4370)
      3. ann_011 (score: 2.4365)
      4. ann_012 (score: 1.1346)
      5. ann_010 (score: 0.9982)
      6. ann_013 (score: 0.9100)
      7. ann_015 (score: 0.7955)
      8. ann_001 (score: 0.5540)
      9. ann_027 (score: 0.2705)
      10. ann_018 (score: 0.1539)


📌 Example 2: 

## Step 10: Best Practices & Optimization Tips

### Recommendations for Production Deployment

In [33]:
print("\n🚀 BEST PRACTICES FOR NEURAL SPARSE SEARCH")
print("="*80)

best_practices = {
    "Category": [
        "Model Selection",
        "Model Selection",
        "Ingestion",
        "Ingestion",
        "Ingestion",
        "Indexing",
        "Indexing",
        "Search",
        "Search",
        "Maintenance"
    ],
    "Best Practice": [
        "Use doc-only mode for most use cases (best balance)",
        "Use bi-encoder mode only for high-relevance requirements",
        "Apply text chunking for documents >512 tokens",
        "Configure pruning ratio (0.1-0.3) to reduce index size",
        "Use ingest pipelines for automatic embedding generation",
        "Set appropriate number_of_shards for parallelism",
        "Exclude embeddings from _source to save disk space",
        "Always specify compatible analyzer for doc-only queries",
        "Combine with filters for better relevance tuning",
        "Monitor model deployment and update as needed"
    ],
    "Impact": [
        "⚡ 2-3x faster queries",
        "📊 +5-10% relevance improvement",
        "💾 Prevents memory issues",
        "💾 30-40% space savings",
        "🔄 Simplified workflow",
        "⚡ Better parallelization",
        "💾 50-60% disk savings",
        "✅ Consistent tokenization",
        "🎯 More precise results",
        "📈 Improved performance"
    ]
}

practices_df = pd.DataFrame(best_practices)
print(practices_df.to_string(index=False))

print("\n\n💡 Pro Tips:")
print("  1. Always test with your actual data before production deployment")
print("  2. Monitor query latency and adjust parameters based on SLA requirements")
print("  3. Use hybrid queries combining sparse + dense search for optimal results")
print("  4. Implement result caching for frequently searched queries")
print("  5. Set up alerts for model deployment failures")


🚀 BEST PRACTICES FOR NEURAL SPARSE SEARCH
       Category                                            Best Practice                         Impact
Model Selection      Use doc-only mode for most use cases (best balance)          ⚡ 2-3x faster queries
Model Selection Use bi-encoder mode only for high-relevance requirements 📊 +5-10% relevance improvement
      Ingestion            Apply text chunking for documents >512 tokens       💾 Prevents memory issues
      Ingestion   Configure pruning ratio (0.1-0.3) to reduce index size         💾 30-40% space savings
      Ingestion  Use ingest pipelines for automatic embedding generation          🔄 Simplified workflow
       Indexing         Set appropriate number_of_shards for parallelism       ⚡ Better parallelization
       Indexing       Exclude embeddings from _source to save disk space          💾 50-60% disk savings
         Search  Always specify compatible analyzer for doc-only queries      ✅ Consistent tokenization
         Search      

## Summary

### What We Covered
1. **Neural Sparse Search Fundamentals** - How sparse embeddings work
2. **Model Registration & Deployment** - Setting up sparse encoding models
3. **Ingest Pipeline Configuration** - Automatic embedding generation
4. **Rank Features Index** - Efficient sparse vector storage
5. **Real-World Use Cases**:
   - E-Commerce Product Search
   - Customer Support Ticket Search
   - Document Repository Search
6. **Performance Optimization** - Neural Sparse ANN for large-scale search
7. **Best Practices** - Production deployment recommendations

### Key Takeaways
✅ Neural Sparse Search achieves the best balance between:
- **Speed** (2-10x faster than dense embeddings)
- **Relevance** (comparable to semantic search)
- **Scalability** (handles millions of documents efficiently)
- **Memory** (low footprint compared to dense methods)

### Next Steps
- Experiment with different sparse encoding models for your domain
- Combine with hybrid queries for further relevance improvements
- Deploy ANN search mode for production at scale
- Monitor and tune based on your specific requirements

In [ ]:
print("\n" + "="*80)
print("✅ Neural Sparse Search Tutorial Complete!")
print("="*80)
print("\n📚 Resources:")
print("  • OpenSearch Docs: https://docs.opensearch.org/latest/vector-search/ai-search/")
print("  • Model Registry: https://docs.opensearch.org/latest/ml-commons-plugin/")
print("  • Performance Tuning: https://docs.opensearch.org/latest/vector-search/performance-tuning-sparse/")
print("\n🎓 Happy Learning!")